In [19]:
#@title Define functions
#@markdown Select model version and run.
from facenet_pytorch import MTCNN
from torchvision import transforms
import torch, PIL
from tqdm.notebook import tqdm
import os

mtcnn = MTCNN(image_size=256, margin=80)

In [20]:
def detect(img):
    #detect faces
    batch_boxes, batch_probs, batch_points = mtcnn.detect(img, landmarks=True)
    #select faces
    if not mtcnn.keep_all:
        batch_boxes, batch_probs, batch_points = mtcnn.select_boxes(
            batch_boxes, batch_probs, batch_points, img, method=mtcnn.selection_method
        )

    return batch_boxes, batch_points

#my version of isOdd, should make a sepatate repo for it :D
def makeEven(_x):
    return _x if (_x % 2 == 0) else _x + 1

#the actual scaler function
def scale(boxes, _img, max_res=1_500_000, target_face=256, fixed_ratio=0, max_upscale=2, VERBOSE=False):
    x, y = _img.size

    ratio = 2   #initial ratio

    #scale to desired face size
    if (boxes is not None):
        if len(boxes) > 0:
            ratio = target_face / max(boxes[0][2:] - boxes[0][:2])
            ratio = min(ratio, max_upscale)
            if VERBOSE:
                print('up by', ratio)

    if fixed_ratio > 0:
        if VERBOSE:
            print('fixed ratio')
        ratio = fixed_ratio

    x *= ratio
    y *= ratio

    #downscale to fit into max res
    res = x * y
    if res > max_res:
        ratio = pow(res / max_res, 1 / 2)
        if VERBOSE:
            print(ratio)
        x = int(x / ratio)
        y = int(y / ratio)

    #make dimensions even, because usually NNs fail on uneven dimensions due skip connection size mismatch
    x = makeEven(int(x))
    y = makeEven(int(y))

    size = (x, y)

    return _img.resize(size)

""" 
    A useful scaler algorithm, based on face detection.
    Takes PIL.Image, returns a uniformly scaled PIL.Image
    boxes: a list of detected bboxes
    _img: PIL.Image
    max_res: maximum pixel area to fit into. Use to stay below the VRAM limits of your GPU.
    target_face: desired face size. Upscale or downscale the whole image to fit the detected face into that dimension.
    fixed_ratio: fixed scale. Ignores the face size, but doesn't ignore the max_res limit.
    max_upscale: maximum upscale ratio. Prevents from scaling images with tiny faces to a blurry mess.
"""

def scale_by_face_size(_img, max_res=1_500_000, target_face=256, fix_ratio=0, max_upscale=2, VERBOSE=False):
    boxes = None
    boxes, _ = detect(_img)
    if VERBOSE:
        print('boxes', boxes)
    img_resized = scale(boxes, _img, max_res, target_face, fix_ratio, max_upscale, VERBOSE)
    return img_resized

size = 256

means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

t_stds = torch.tensor(stds).cuda().half()[:, None, None]
t_means = torch.tensor(means).cuda().half()[:, None, None]

def makeEven(_x):
    return int(_x) if (_x % 2 == 0) else int(_x + 1)

img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(means, stds)
])

def tensor2im(var):
    return var.mul(t_stds).add(t_means).mul(255.).clamp(0, 255).permute(1, 2, 0)

def proc_pil_img(input_image, model):
    transformed_image = img_transforms(input_image)[None, ...].cuda().half()

    with torch.no_grad():
        result_image = model(transformed_image)[0]; print(result_image.shape)
        output_image = tensor2im(result_image)
        output_image = output_image.detach().cpu().numpy().astype('uint8')
        output_image = PIL.Image.fromarray(output_image)
    return output_image





#load model
#模型路劲
model_path = 'D:/paddle/argan/ArcaneGANv0.4.jit'
#图片输入路劲
in_dir = 'D:/paddle/argan/c.jpg'
#图片保存路劲
out_dir = 'D:/paddle/argan/c_result.jpg'









model = torch.jit.load(model_path).eval().cuda().half()

def fit(img, maxsize=512):
    maxdim = max(*img.size)
    if maxdim > maxsize:
        ratio = maxsize / maxdim
        x, y = img.size
        size = (int(x * ratio), int(y * ratio))
        img = img.resize(size)
    return img

def show_img(f, size=1024):
    display(fit(PIL.Image.open(f), size))

def process(upload=True):
    img = in_dir
    im = PIL.Image.open(img)
    im = scale_by_face_size(im, target_face=300, max_res=1_500_500, max_upscale=2)
    res = proc_pil_img(im, model)
    res.save(out_dir)

In [21]:
process()

torch.Size([3, 1028, 770])
